In [58]:
import os
import pandas as pd
import gmaps
import googlemaps as g
import yaml
import numpy
import matplotlib.pyplot as plt
from IPython.display import Image
import foursquare



%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.notebook_repr_html', True)

# Loading dataset
df = pd.read_csv(os.path.join('..', 'datasets', 'avg_inspection_scores_by_venue.csv'))
# df = df.set_index(id)
df.drop(['Unnamed: 0'],inplace = True, axis = 1)
df.head()

,business_name,location,inspection_score
0,Tiramisu Kitchen,"(37.791116, -122.403816)",87.400000
1,Nrgize Lifestyle Cafe,"(37.786848, -122.421547)",94.000000
2,OMNI S.F. Hotel - 2nd Floor Pantry,"(37.792888, -122.403135)",96.333333
3,Norman's Ice Cream and Freezes,"(37.807155, -122.419004)",87.750000
4,CHARLIE'S DELI CAFE,"(37.747114, -122.413641)",83.200000


In [59]:
len(df)

5568

In [71]:
# Applying foursquare credentials
with open('foursquare.yaml', 'r') as f:
    fs_credentials = yaml.load(f)

fs_client_id = fs_credentials['client-id']
fs_client_secret = fs_credentials['client-secret']



In [69]:
client = foursquare.Foursquare(client_id=fs_client_id, client_secret=fs_client_secret)

In [70]:
def fs_idlookup(name, latlong):
    location_str = latlong.lstrip('(').rstrip(')')
    object =  client.venues.search(params={'query': name, 'll': location_str, 'limit': 1, 'intent': '"match"' }) 
    if object['venues']:
        return (object['venues'][0]['id'], object['venues'][0]['name'])
    else:
        print(object)
        print("Could not find {0} at {1}".format(name, latlong))

print fs_idlookup(df.business_name[0], df.location[0])
# client.venues.search(params={'query': 'Tiramisu Kitchen', 'll': '37.791116, -122.403816', 'limit': 1, 'intent': 'match' })

(u'4ae289b9f964a520e78e21e3', u'Cafe Tiramisu')


In [64]:
fs_ids = {}
fs_names = {}
for i in range(0,10):
    result = fs_idlookup(df.business_name[i], df.location[i])
    if result:
        id, name = result
        fs_ids[i] = id
        fs_names[i] = name
    
df['foursquare_id'] = pd.Series(fs_ids)
df['foursquare_name'] = pd.Series(fs_names)

{u'venues': []}
Could not find OMNI S.F. Hotel - 2nd Floor Pantry at (37.792888, -122.403135)


In [65]:
df.head()

,business_name,location,inspection_score,foursquare_id,foursquare_name
0,Tiramisu Kitchen,"(37.791116, -122.403816)",87.400000,4ae289b9f964a520e78e21e3,Cafe Tiramisu
1,Nrgize Lifestyle Cafe,"(37.786848, -122.421547)",94.000000,4c1a7695eac020a1105043c2,Nrgize Lifestyle Cafe
2,OMNI S.F. Hotel - 2nd Floor Pantry,"(37.792888, -122.403135)",96.333333,NaN,NaN
3,Norman's Ice Cream and Freezes,"(37.807155, -122.419004)",87.750000,4bb7d9273db7b71353aa209a,Norman's Ice Cream & Freezes
4,CHARLIE'S DELI CAFE,"(37.747114, -122.413641)",83.200000,4ab90c8bf964a520d27d20e3,Charlie's Deli Cafe


In [66]:
df.to_csv(os.path.join('..','datasets','foursquare_idmapping.csv'))